In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell, seq2seq
from tqdm import *
from tensorflow.python.ops import control_flow_ops
import sys
import random
import time

from Utils import Utils
import cnn_lstm_lstm as model
from hyperboard import Agent

    
def train(train_utils, valid_utils, source_len, oseq_len, n_echos, simplified_len, decoder_hidden, encoder_hidden,
          embedding_size, batch_size, display_step, prob, lstm_layer, learning_rate,
          model_dir, source_nfilters, source_width):
            
    
    hyperparameters = {
        'learning rate': learning_rate,
        'batch size': batch_size,
        'criteria': '',
        'decoder hidden': decoder_hidden,
        'encoder_hidden':encoder_hidden,
        'prob': prob,
        'source_nfilters': source_nfilters,
        'source_width': source_width,
        'lstm_layer': lstm_layer,  
        'embedding_size': embedding_size
    }
        
    agent = Agent(port = 5000)
    
    hyperparameters['criteria'] = 'valid loss'
    name_valid_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'valid accu'
    name_valid_accu = agent.register(hyperparameters, 'accuracy')
    
    hyperparameters['criteria'] = 'train loss'
    name_train_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'train accu'
    name_train_accu = agent.register(hyperparameters, 'accuracy')
    
        
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
        
    words_size = train_utils.get_words_size()
    
    outputs = model.build_model(words_size=words_size, 
                                embedding_size=embedding_size,
                                source_len=source_len,
                                simplified_len=simplified_len,
                                oseq_len=oseq_len, 
                                decoder_hidden=decoder_hidden,
                                encoder_hidden=encoder_hidden,
                                source_nfilters=source_nfilters,
                                source_width=source_width,
                                lstm_layer=lstm_layer, 
                                batch_size=batch_size, 
                                is_train=True)

    cost=outputs['cost']
    words_prediction=outputs['words_prediction']
    source=outputs['source']
    defendant=outputs['defendant']
    defendant_length=outputs['defendant_length']
    label=outputs['label']
    decoder_inputs=outputs['decoder_inputs']
    loss_weights=outputs['loss_weights']
    keep_prob=outputs['keep_prob']
    sample_rate=outputs['sample_rate']
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='adam_optimizer')
    # Compute the gradients for a list of variables.
    grads_and_vars = optimizer.compute_gradients(cost)
    # grads_and_vars is a list of tuples (gradient, variable).
    capped_grads_and_vars = [(tf.clip_by_norm(g, 5), v) for g,v in grads_and_vars]
    # Ask the optimizer to apply the capped gradients.
    train_op = optimizer.apply_gradients(capped_grads_and_vars)

    
    init = tf.initialize_all_variables()

    os.environ["CUDA_VISIBLE_DEVICES"] = '1'
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction = 0.7)
    session_conf = tf.ConfigProto(allow_soft_placement = True, 
                                  log_device_placement = False,
                                  gpu_options = gpu_option)
    
    sess = tf.Session()
    
#     pre_saver = tf.train.Saver()
#     ckpt = tf.train.latest_checkpoint('/home/xuwenshen/2017_3_13/cnn_lstm-lstm/model_v1')
#     pre_saver.restore(sess, ckpt)
    
    global_steps = 0
    
    echos = 0
    batch = 0
    
    saver = tf.train.Saver(max_to_keep = 20)

    sess.run(init)
    
    tvar = tf.trainable_variables()
    for v in tvar:
        print (v.name)
    
    print ('init done')
    
    train_cost = 0
    train_accu = 0
    train_bleu = 0
    
    while True:
        
        datas, is_again = train_utils.next_batch()
        
        if is_again:
            
            train_cost = 0
            train_accu = 0
            echos += 1
            batch = 0
            if echos == n_echos:
                break
            continue
            
        
        batch_source = datas['source'] 
        batch_defendant = datas['defendant'] 
        batch_defendant_length = datas['defendant_length']
        batch_ground_truth = datas['ground_truth']
        batch_label = datas['label']
        batch_weights = datas['loss_weights']
        
        batch += 1
        

        sample_rate_ = min(.5, 0.01 * (global_steps))

        feed_dic = {source:batch_source,
                    defendant:batch_defendant,
                    label:batch_label,
                    defendant_length:batch_defendant_length,
                    decoder_inputs:batch_ground_truth,
                    loss_weights:batch_weights,
                    keep_prob:prob,
                    sample_rate:sample_rate_}
        
        words_, cost_, _= sess.run([words_prediction, cost, train_op], feed_dict= feed_dic)
#         accu_, bleu_ = train_utils.i2t(words_, to_print = False)
    
        train_cost += cost_
#         train_accu += accu_
#         train_bleu += bleu_
        

        
        if batch % display_step == 0:
            
            valid_cost = 0
            valid_accu = 0
            valid_bleu = 0
            valid_batchs = 0
            
            train_cost /= display_step
            train_accu /= display_step
            train_bleu /= display_step
            
            while True:
                
                datas, is_again = valid_utils.next_batch()
                
                if is_again:
                    break
        
                batch_source = datas['source'] 
                batch_defendant = datas['defendant'] 
                batch_defendant_length = datas['defendant_length']
                batch_ground_truth = datas['ground_truth']
                batch_label = datas['label']
                batch_weights = datas['loss_weights']

                valid_batchs += 1
                
                feed_dic = {source:batch_source,
                            defendant:batch_defendant,
                            label:batch_label,
                            defendant_length:batch_defendant_length,
                            decoder_inputs:batch_ground_truth,
                            loss_weights:batch_weights,
                            keep_prob:prob,
                            sample_rate:sample_rate_}
                    
                word_, cost_, = sess.run([words_prediction, cost], feed_dict=feed_dic)     
                accu_, bleu_ = valid_utils.i2t(word_, to_print = True)
                
                valid_cost += cost_
                valid_accu += accu_
                valid_bleu += bleu_
            
            valid_cost /= valid_batchs
            valid_accu /= valid_batchs
            valid_bleu /= valid_batchs
            
            saver.save(sess, model_dir + \
                       "sample_rate-" + str(sample_rate_) + \
                       "-train_accu-{:.3f}".format(train_accu) + \
                       "-train_cost-{:.3f}".format(train_cost) + \
                       "-train_bleu-{:.3f}".format(train_bleu) + \
                       "-valid_accu-{:.3f}".format(valid_accu) + \
                       "-valid_cost-{:.3f}".format(valid_cost) + \
                       "-valid_bleu-{:.3f}".format(valid_bleu) + \
                       "-model.ckpt", global_step = global_steps)
            
            
            print ("Echo: "+str(echos) + " Iters: "+str(batch) + \
                   " Sample: " + "{:.3f}".format(sample_rate_) + \
                   " Train loss: " + "{:.3f}".format(train_cost) + \
                   " Valid loss: " + "{:.3f}".format(valid_cost) + \
                   " Train accu: " + "{:.3f}".format(train_accu) + \
                   " Valid accu: " + "{:.3f}".format(valid_accu) + \
                   " Train bleu: " + "{:.3f}".format(train_bleu) + \
                   " Valid bleu: " + "{:.3f}".format(valid_bleu))
            

            agent.append(name_valid_loss, global_steps, valid_cost)
            agent.append(name_valid_accu, global_steps, valid_accu)
            agent.append(name_train_loss, global_steps, train_cost)
            agent.append(name_train_accu, global_steps, train_accu)
            
            global_steps += 1
            
        
            
if __name__ == '__main__':
    
    words_path = '/home/xuwenshen/data/big_data/2017_3_13/words'
    train_path = '/home/xuwenshen/data/big_data/2017_3_13/train.h5'
    valid_path = '/home/xuwenshen/data/big_data/2017_3_13/valid.h5'
    
    oseq_len = 200
    source_len = 1000
    simplified_len = 150

    batch_size = 64
    display_step = 400
    n_echos = 100
    prob = 0.75
    decoder_hidden = 400
    encoder_hidden = 400
    source_nfilters = 128
    source_width = 3
    lstm_layer = 1
    learning_rate = 0.001
    embedding_size = 200


    
    
    train_utils = Utils(words_path, train_path, batch_size, nb_samples=1600000)
    valid_utils = Utils(words_path, valid_path, batch_size, nb_samples=640)

    train(train_utils=train_utils,
          valid_utils=valid_utils,
          source_len=source_len,
          simplified_len=simplified_len,
          oseq_len=oseq_len,
          embedding_size=embedding_size,
          batch_size=batch_size,
          display_step=display_step,
          n_echos=n_echos,
          prob=prob,
          decoder_hidden=decoder_hidden,
          encoder_hidden=encoder_hidden,
          source_nfilters=source_nfilters,
          source_width=source_width,
          lstm_layer=lstm_layer,
          learning_rate=learning_rate,
          model_dir='/home/xuwenshen/2017_3_13/cnn_lstm-lstm/model_v1/')
    
    
    

overwrite? [Y/n]y
overwrite? [Y/n]y
overwrite? [Y/n]y
overwrite? [Y/n]y
[None, 998, 1, 128]
[None, 996, 1, 128]
[None, 994, 1, 128]
build model 
embedding:0
conv_source_w_1:0
conv_source_w_2:0
conv_source_w_3:0
conv_source_b_1:0
conv_source_b_2:0
conv_source_b_3:0
conv_source_scale_1:0
conv_source_scale_2:0
conv_source_scale_3:0
conv_source_offset_1:0
conv_source_offset_2:0
conv_source_offset_3:0
generation_w:0
generation_b:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Matrix:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Bias:0
rnn_decoder/BasicLSTMCell/Linear/Matrix:0
rnn_decoder/BasicLSTMCell/Linear/Bias:0
init done
公诉机关浙江省淳安县人民检察院。被告人樊某甲。2007年5月13日因本案被淳安县公安局刑事拘留，同年6月19日被依法逮捕。现押于淳安县看守所。辩护人王绍基。被告人樊某乙。2007年5月13日因本案被淳安县公安局刑事拘留，同年6月19日被依法逮捕。现押于淳安县看守所。辩护人章献彪。被告人樊某丙。2007年5月13日因本案被淳安县公安局刑事拘留，同年6月19日被依法逮捕。现押于淳安县看守所。辩护人余建友。淳安县人民检察院以淳检刑诉（2007）247号起诉书指控被告人樊某甲、樊某乙、樊某丙犯寻衅滋事罪，于2007年9月17日向本院提起公诉。本院受理后，依法适用简易程序，实行独任审判，公开开庭审理了本案。被告人樊某甲、樊某乙、樊某丙及辩护人王绍基、章献彪、余建友到庭参加诉讼。现已审理终结。淳安县人民检察院起诉指控：2007年3月份的一天，被害人涂志清到淳安县

/home/xuwenshen/py3/lib/python3.4/site-packages/nltk/translate/bleu_score.py:472: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


公诉机关湖北省阳新县人民检察院。被告人陈某，男。因涉嫌犯危险驾驶罪于2015年12月1日经本院决定被执行逮捕，现羁押于阳新县看守所。湖北省阳新县人民检察院以阳检刑诉（2015）260号起诉书指控被告人陈某犯危险驾驶罪，于2015年12月1日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。阳新县人民检察院指派代理检察员王聪出庭支持公诉，被告人陈某到庭参加诉讼。现已审理终结。阳新县人民检察院指控，2015年11月3日下午4时许，被告人陈某在阳新县兴国镇林峰路“口堂”餐馆聚餐饮酒后，驾驶鄂某号新大洲本田二轮摩托车载乘妻子和女儿返回本县经济开发区官桥村。该车行至城北工业园金三角世纪城门前路段时与横穿公路的刘某发生碰撞，造成刘某受伤的事故。经理化鉴定，被告人陈某血液中的乙醇含量为123.4mg／100ml。经交警部门认定，陈某负事故的全部责任。案发后，被告人陈某除支付全部医药费外另额外赔偿被害人损失人民币5000元，取得了谅解。上述事实，被告人陈某在庭审中亦无异议，且有到案经过、驾驶人、机动车信息查询单、血液提取登记表、案发现场照片、协议书、谅解书以及户籍证明等书证，证人王定、刘某的证言，理化鉴定书、道路交通事故认定书，讯问同步录音录像等证据证实，足以认定。
--------------------

被告人陈某，260号起诉书指控被告人陈某犯危险驾驶罪，被告人陈某到庭参加诉讼，被告人陈某在阳新县兴国镇林峰路“口堂”餐馆聚餐饮酒后，被告人陈某血液中的乙醇含量为123.4mg／100ml，被告人陈某在庭审中亦无异议，因涉嫌犯危险驾驶罪于2015年12月1日经本院决定被执行逮捕，被告人陈某除支付全部医药费外另额外赔偿被害人损失人民币5000元，
--------------------

本院认为，被告人陈某醉酒后在道路上驾驶机动车辆，其行为已构成危险驾驶罪。公诉机关指控的罪名成立。被告人陈某醉酒驾驶机动车辆并发生交通事故，且负事故全部责任，酌情予以从重处罚。但其归案后如实供述自己罪行，当庭自愿认罪，并已经对被害人进行了赔偿，得到了谅解，可酌情从轻处罚
--------------------

本院认为，被告人李某某酒驾驾道路上醉驶机动车，，其行为已构成危险驾驶罪。公诉机关指控的罪名成立。被告人李某某酒后驶机动车，，取生产通事实，其其事实意部责任，其情从